In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.model_solver import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

def build_approx_theta(prf, init_theta = None):
    connivents = []
    if not init_theta:
        init_theta = [EMPTY_SET]
    theta = init_theta
    min_k = 1
    c  = get_connivent(theta, prf)
    while c:
        if not c in connivents:
            connivents.append(c)
        cit = get_candidate_iterator(c)
        skey = sorted(cit.keys())[0]
        b = False
        for k in cit:
            if b:
                break
            for i in cit[k]:
                for t in i:
                    b = False or check_connivence_resolution(c, t)
                    if not t in theta and check_connivence_resolution(c, t):
                        theta.append(t)
        c  = get_connivent(theta, prf)
    #a = additivity(theta)
    #for c_i in connivents:
    #    cit = get_candidate_iterator(c_i)
    #    for k in cit:
    #        if k > a:
    #            break
    #        for i in cit[k]:
    #            for t in i:
    #                if not t in theta and check_connivence_resolution(c_i,t):
    #                    theta.append(t)
    #
    return theta

In [2]:
fd = Film_Dataset(60,9)
prf = fd.get_preferences_items(8)
t_heuristic = build_approx_theta(prf, [EMPTY_SET])
t_heuristic

[(2,),
 (3,),
 (0, 3),
 (3, 4),
 (3, 7),
 (3, 9),
 (3, 11),
 (3, 13),
 (3, 15),
 (0, 2),
 (2, 4),
 (2, 7),
 (2, 9),
 (2, 11),
 (2, 13),
 (2, 15),
 (0, 3, 4),
 (0, 3, 7),
 (0, 3, 9),
 (0, 3, 11),
 (0, 3, 13),
 (0, 3, 15),
 (3, 4, 7),
 (3, 4, 9),
 (3, 4, 11),
 (3, 4, 13),
 (3, 4, 15),
 (3, 7, 9),
 (3, 7, 11),
 (3, 7, 13),
 (3, 7, 15),
 (3, 9, 11),
 (3, 9, 13),
 (3, 9, 15),
 (3, 11, 13),
 (3, 11, 15),
 (3, 13, 15),
 (0, 2, 4),
 (0, 2, 7),
 (0, 2, 9),
 (0, 2, 11),
 (0, 2, 13),
 (0, 2, 15),
 (2, 4, 7),
 (2, 4, 9),
 (2, 4, 11),
 (2, 4, 13),
 (2, 4, 15),
 (2, 7, 9),
 (2, 7, 11),
 (2, 7, 13),
 (2, 7, 15),
 (2, 9, 11),
 (2, 9, 13),
 (2, 9, 15),
 (2, 11, 13),
 (2, 11, 15),
 (2, 13, 15),
 (0, 2, 4, 7),
 (0, 2, 4, 9),
 (0, 2, 4, 11),
 (0, 2, 4, 13),
 (0, 2, 4, 15),
 (0, 2, 7, 9),
 (0, 2, 7, 11),
 (0, 2, 7, 13),
 (0, 2, 7, 15),
 (0, 2, 9, 11),
 (0, 2, 9, 13),
 (0, 2, 9, 15),
 (0, 2, 11, 13),
 (0, 2, 11, 15),
 (0, 2, 13, 15),
 (2, 4, 7, 9),
 (2, 4, 7, 11),
 (2, 4, 7, 13),
 (2, 4, 7, 15),
 (2, 4, 9, 

In [ ]:
t_mins = get_kernels_lex3(prf,t_heuristic)
t_mins

Found n°1 =  [(2,), (0, 2), (2, 4), (0, 2, 4), (0, 2, 15), (2, 4, 9), (7, 16), (14, 16)] with obj: 42
Found n°2 =  [(2,), (0, 2), (2, 4), (0, 2, 4), (0, 2, 15), (2, 4, 9), (7, 16), (8, 16)] with obj: 42
Found n°3 =  [(2,), (0, 2), (2, 7), (0, 2, 13), (2, 9, 15), (2, 13, 15), (15, 16), (6, 16)] with obj: 42
Found n°4 =  [(3,), (0, 2), (2, 13), (0, 3, 4), (2, 9, 15), (2, 13, 15), (15, 16), (6, 16)] with obj: 42
